In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
import warnings

warnings.filterwarnings("ignore")
from flaml import logger, logging

logger.setLevel(logging.WARNING)
import random

imp = pd.read_csv("Imputed_scaled.csv")
miss = pd.read_csv("Missingness_0_scaled.csv")
smote = pd.read_csv("Smote_scaled.csv")
TEST = pd.read_csv("unlabeled.csv")

cols = imp.columns
cols = cols.drop(cols[0])

test_X = TEST[cols]
testing_X = test_X.drop("hospital_death", axis=1)

imp = imp[cols]
miss = miss.drop(miss.columns[0], axis=1)
smote = smote.drop(smote.columns[0], axis=1)

smote_X = smote.drop("hospital_death", axis=1)
smote_y = smote["hospital_death"]
random.seed(101)
smote_x_train, smote_x_test, smote_y_train, smote_y_test = train_test_split(
    smote_X, smote_y, test_size=0.3, random_state=47
)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0, scale=(3 / 171)),  # number of features
}

rbf_cv = RandomizedSearchCV(
    svm.SVC(random_state=13, kernel="rbf"),
    dists,
    n_iter=10,
    n_jobs=-1,
    scoring=["accuracy", "f1", "precision", "recall"],
    refit="accuracy",
    cv=5,
    random_state=14,
    verbose=0,
)

rbf_cv.fit(smote_x_train, smote_y_train)

rbf_cv_pred = rbf_cv.predict(smote_x_test)
rbf_cv_sc = rbf_cv.score(smote_x_test, smote_y_test)
rbf_cv_results = pd.DataFrame(rbf_cv.cv_results_)

print(rbf_cv_sc)

pd.set_option('display.max_columns', None)
rbf_cv_results.loc[rbf_cv_results['rank_test_accuracy']==1]


0.8160965261203925


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
3,26.975872,0.188141,4.960639,0.068036,1.53935,0.005584,"{'C': 1.5393501368036615, 'gamma': 0.005583937...",0.826087,0.830918,0.818414,0.822393,0.815288,0.82262,0.00552,1,0.712946,0.723384,0.703755,0.711316,0.694836,0.709247,0.009546,5,0.793319,0.796315,0.772126,0.77621,0.77325,0.782244,0.010396,2,0.647359,0.662692,0.646508,0.656436,0.630861,0.648771,0.010774,6


In [ ]:

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")  # another: median
test_trunc_x_imp = imputer.fit_transform(testing_X)

rbf_cv_pred_test = rbf_cv.predict(test_trunc_x_imp)
rbf_cv_results_test = pd.DataFrame(rbf_cv.cv_results_)

rbf_cv_pred_test0 = rbf_cv.predict(testing_X.fillna(0))
rbf_cv_results_test0 = pd.DataFrame(rbf_cv.cv_results_)

In [ ]:

rbf_cv.fit(smote_x_train, smote_y_train)

rbf_cv_pred = rbf_cv.predict(smote_x_test)
rbf_cv_sc = rbf_cv.score(smote_x_test, smote_y_test)
rbf_cv_results = pd.DataFrame(rbf_cv.cv_results_)

print(rbf_cv_sc)

print(rbf_cv_results)

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")  # another: median
test_trunc_x_imp = imputer.fit_transform(testing_X)

rbf_cv_pred_test = rbf_cv.predict(test_trunc_x_imp)
rbf_cv_results_test = pd.DataFrame(rbf_cv.cv_results_)

rbf_cv_pred_test0 = rbf_cv.predict(testing_X.fillna(0))
rbf_cv_results_test0 = pd.DataFrame(rbf_cv.cv_results_)